In [7]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import numpy as np
import pandas as pd
import math
import glob
import time
import pickle

import steering
import speeding

import logging

In [14]:
PROJECT_ROOT = '../../../..'
LEARNED_DRIVER = 'default_miner/data'
DIFFICULTY = 'easy'

# COMMAND = 'steering'
COMMAND = 'speeding'
logging.basicConfig(filename='logs/{}/training-{}.log'.format(COMMAND, time.time()),level=logging.DEBUG)

NUM_EPOCHS = 50

TRAINING_FILES = glob.glob('/'.join([PROJECT_ROOT, LEARNED_DRIVER, DIFFICULTY, '*.csv']))
TRAINING_DATA = {}
for FILE in TRAINING_FILES:
    DF = pd.read_csv(FILE, index_col=False)
    TRAINING_DATA[FILE] = DF.values

VALIDATION_FILES = glob.glob('/'.join([PROJECT_ROOT, LEARNED_DRIVER, 'validation', '*.csv']))
VALIDATION_DATA = {}
for FILE in VALIDATION_FILES:
    DF = pd.read_csv(FILE, index_col=False)
    VALIDATION_DATA[FILE] = DF.values
    
CUDA = torch.cuda.is_available()
if CUDA:
    DTYPE = torch.cuda.FloatTensor
else:
    DTYPE = torch.FloatTensor

In [15]:
def save_checkpoint(state, is_best, filepath='latest_checkpoint.tar'):
    torch.save(state, 'split_checkpoints/' + DIFFICULTY[-1] + '/' + COMMAND + '/' + filepath)
    if is_best:
        torch.save(state, 'split_checkpoints/' + DIFFICULTY[-1] + '/' + COMMAND + '/' + 'best_checkpoint.tar')

In [17]:
def train(training_data, model, criterion, optimzier):
    loss = 0
    model.train(mode=True)
    for key in training_data:
        logging.info('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        print('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        
        model.hidden_state = model.init_hidden()
        optimizer.zero_grad()
        track_sequence = training_data[key]

        if COMMAND == 'steering':
            targets = track_sequence[:, 2:3]
        elif COMMAND == 'speeding':
            targets = track_sequence[:, 0:2]
        
        inputs = track_sequence[:, 3:]

        targets_variable = autograd.Variable(torch.Tensor(targets)).type(DTYPE)
        inputs_variable = autograd.Variable(torch.Tensor(inputs),  requires_grad=True).type(DTYPE)

        outputs_variable = model(inputs_variable)

        track_loss = criterion(outputs_variable, targets_variable)

        track_loss.backward()
        optimizer.step()

        loss += track_loss.data[0]
    return loss

In [18]:
def validate(validation_data, model, criterion):
    loss = 0
    model.train(mode=False)
    for key in validation_data:
        logging.info('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        print('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        
        model.hidden_state = model.init_hidden()
        track_sequence = validation_data[key]

        if COMMAND == 'steering':
            targets = track_sequence[:, 2:3]
        elif COMMAND == 'speeding':
            targets = track_sequence[:, 0:2]
            
        inputs = track_sequence[:, 3:]

        targets_variable = autograd.Variable(torch.Tensor(targets), volatile=True).type(DTYPE)
        inputs_variable = autograd.Variable(torch.Tensor(inputs), volatile=True).type(DTYPE)

        outputs_variable = model(inputs_variable)

        track_loss = criterion(outputs_variable, targets_variable)

        loss += track_loss.data[0]
    return loss

In [19]:
if COMMAND == 'steering':
    model = steering.Steering(steering.HYPERPARAMS.INPUT_SIZE,
                               steering.HYPERPARAMS.LSTM_HIDDEN_SIZE,
                               steering.HYPERPARAMS.HIDDEN_LAYER_SIZE,
                               steering.HYPERPARAMS.DROPOUT_PROB,
                               steering.HYPERPARAMS.NUM_LAYERS,
                               steering.HYPERPARAMS.TARGET_SIZE,
                               steering.HYPERPARAMS.BATCH_SIZE)
    optimizer = optim.Adam(model.parameters(), lr=steering.HYPERPARAMS.LEARNING_RATE)
elif COMMAND == 'speeding':
    model = speeding.Speeding(speeding.HYPERPARAMS.INPUT_SIZE,
                               speeding.HYPERPARAMS.LSTM_HIDDEN_SIZE,
                               speeding.HYPERPARAMS.HIDDEN_LAYER_SIZE,
                               speeding.HYPERPARAMS.DROPOUT_PROB,
                               speeding.HYPERPARAMS.NUM_LAYERS,
                               speeding.HYPERPARAMS.TARGET_SIZE,
                               speeding.HYPERPARAMS.BATCH_SIZE)
    optimizer = optim.Adam(model.parameters(), lr=speeding.HYPERPARAMS.LEARNING_RATE)
if CUDA:
    model.cuda()

criterion = nn.MSELoss().cuda()
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', verbose=True, factor=0.05)

min_loss = math.inf
losses = {
  'training': [],
  'validation': []
}

In [22]:
logging.info('Training %s...' % COMMAND)
print('Training %s...' % COMMAND)

for epoch in np.arange(NUM_EPOCHS):
    logging.info('Epoch [%d/%d]' %(epoch+1, NUM_EPOCHS))
    print('Epoch [%d/%d]' %(epoch+1, NUM_EPOCHS))
    
    is_best = False

    training_loss = train(TRAINING_DATA, model, criterion, optimizer)
    logging.info('--- TRAINING LOSS: %f' % training_loss)
    print('--- TRAINING LOSS: %f' % training_loss)

#     validation_loss = validate(VALIDATION_DATA, model, criterion)
#     logging.info('--- VALIDATION LOSS: %f' % validation_loss)
#     print('--- VALIDATION LOSS: %f' % validation_loss)

    if training_loss < min_loss:
        logging.info('--- --- best model found so far: %f' % training_loss)
        print('--- --- best model found so far: %f' % training_loss)
        min_loss = training_loss
        is_best = True

    losses['training'].append(training_loss)
#     losses['validation'].append(validation_loss)

    save_checkpoint({
          'epoch': epoch + 1,
          'state_dict': model.state_dict(),
          'min_loss': min_loss,
          'optimizer' : optimizer.state_dict(),
      }, is_best)

    scheduler.step(training_loss)
    logging.info('-------------------------------------------------------')
    print('-------------------------------------------------------')

Training speeding...
Epoch [1/50]
--- Parsing track easy-race_CDRV.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track easy-race_QDWP.csv
--- Parsing track easy-race_PVDY.csv
--- TRAINING LOSS: 0.054986
--- --- best model found so far: 0.054986
-------------------------------------------------------
Epoch [2/50]
--- Parsing track easy-race_CDRV.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track easy-race_QDWP.csv
--- Parsing track easy-race_PVDY.csv
--- TRAINING LOSS: 0.028637
--- --- best model found so far: 0.028637
-------------------------------------------------------
Epoch [3/50]
--- Parsing track easy-race_CDRV.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track easy-race_QDWP.csv
--- Parsing track easy-race_PVDY.csv
--- TRAINING LOSS: 0.019505
--- --- best model found so far: 0.019505
-------------------------------------------------------
Epoch [4/50]
--- Parsing track easy-race_CDRV.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track easy-race_QDW

--- Parsing track easy-race_PVDY.csv
--- TRAINING LOSS: 0.012325
-------------------------------------------------------
Epoch [31/50]
--- Parsing track easy-race_CDRV.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track easy-race_QDWP.csv
--- Parsing track easy-race_PVDY.csv
--- TRAINING LOSS: 0.008977
-------------------------------------------------------
Epoch [32/50]
--- Parsing track easy-race_CDRV.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track easy-race_QDWP.csv
--- Parsing track easy-race_PVDY.csv
--- TRAINING LOSS: 0.006894
-------------------------------------------------------
Epoch [33/50]
--- Parsing track easy-race_CDRV.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track easy-race_QDWP.csv
--- Parsing track easy-race_PVDY.csv
--- TRAINING LOSS: 0.005686
-------------------------------------------------------
Epoch [34/50]
--- Parsing track easy-race_CDRV.csv
--- Parsing track easy-race_UXES.csv
--- Parsing track easy-race_QDWP.csv
--- Parsing trac

In [20]:
# dump losses to file
with open('split_checkpoints/{}/losses_{}.pkl'.format(COMMAND, time.time()), 'wb') as file:
    pickle.dump(losses, file)

In [21]:
# test dump worked
with open('split_checkpoints/{}/losses.pkl'.format(COMMAND, time.time()), 'rb') as file:
    hello = pickle.load(file)
hello

{'training': [], 'validation': []}